# Doctors: AI-Powered Quality Analysis

Doctors analyze AI artifacts (prompts, tools, MCP servers, agent cards) and provide recommendations.
They use an LLM to evaluate quality, clarity, and completeness.

In [ ]:
from agentic_patterns.core.doctors.prompt_doctor import PromptDoctor
from agentic_patterns.core.doctors.tool_doctor import ToolDoctor
from agentic_patterns.core.doctors.mcp_doctor import MCPDoctor
from agentic_patterns.core.doctors.a2a_doctor import A2ADoctor
from pydantic_ai.mcp import MCPServerStdio

## PromptDoctor: Analyzing Prompts

PromptDoctor evaluates prompt templates for clarity, completeness, and potential issues.

In [ ]:
# A poorly written prompt
bad_prompt = "do the thing with {x}"

doctor = PromptDoctor()
result = await doctor.analyze(bad_prompt)
print(result)

In [ ]:
# A well-written prompt
good_prompt = """You are a technical writer. Summarize the following document in 3 bullet points.

Document:
{document}

Respond with exactly 3 bullet points, each starting with a dash."""

result = await doctor.analyze(good_prompt)
print(result)

## ToolDoctor: Analyzing Tool Functions

ToolDoctor evaluates tool definitions for proper naming, documentation, and type hints.

In [ ]:
# A poorly defined tool
def do_stuff(x):
    """Does stuff."""
    return x

doctor = ToolDoctor()
result = await doctor.analyze(do_stuff)
print(result)

In [ ]:
# A well-defined tool
def search_database(query: str, limit: int = 10) -> list[dict]:
    """Search the database for records matching the query.
    
    Returns a list of matching records, each containing 'id', 'name', and 'score' fields.
    """
    return []

result = await doctor.analyze(search_database)
print(result)

## MCPDoctor: Analyzing MCP Server Tools

MCPDoctor connects to an MCP server and analyzes all its exposed tools.
We have two example servers: `mcp_server_bad.py` (poorly defined) and `mcp_server_good.py` (well defined).

In [ ]:
# Analyze the poorly defined MCP server
mcp_server = MCPServerStdio(command="fastmcp", args=["run", "-t", "stdio", "mcp_server_bad.py"])

doctor = MCPDoctor(mcp_server)
results = await doctor.analyze_all()

for result in results:
    print(result)
    print()

In [ ]:
# Analyze the well-defined MCP server
mcp_server = MCPServerStdio(command="fastmcp", args=["run", "-t", "stdio", "mcp_server_good.py"])

doctor = MCPDoctor(mcp_server)
results = await doctor.analyze_all()

for result in results:
    print(result)
    print()

## A2ADoctor: Analyzing Agent Cards

A2ADoctor connects to A2A servers and analyzes their agent cards.

Start the servers in separate terminals:

```bash
# In termina 1
cd agentic_patterns/examples/evals
uvicorn a2a_server_bad:app --port 8001

# In termina 2
cd agentic_patterns/examples/evals
uvicorn a2a_server_good:app --port 8002
```

In [ ]:
# Analyze the poorly defined A2A server
doctor = A2ADoctor()
result = await doctor.analyze("http://127.0.0.1:8001")
print(result)

In [ ]:
# Analyze the well-defined A2A server
result = await doctor.analyze("http://127.0.0.1:8002")
print(result)